In [55]:
import pandas as pd
import parser
import imp
import pickle

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100
imp.reload(parser)

SyntaxError: EOL while scanning string literal (parser.py, line 14)

In [ ]:
!git commit -a -m "minor fix"

[master 9e5d6f8] minor fix
 2 files changed, 8 insertions(+), 61 deletions(-)


In [27]:
dataset = pd.read_excel("np_ngo_classification.xlsx")
dataset.head()

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Ссылка на соц сеть,Вид деятельности 1,...,Вид деятельности 3,Вид деятельности 4,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12
0,Москва,"Межрегиональная общественная благотворительная организация ""Общество защиты прав потребителей и ...",5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-й км, д 6",0,0,"http://greenstartup.ru/, http:/9911850.ru/",https://princip1.livejournal.com/,3_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Москва,"Межрегиональная благотворительная общественная организация инвалидов ""Общество пациентов с перви...",7702370774,2007-08-28,"г Москва, Мещанский р-н, ул Трифоновская, д 49, оф 41",1,1,http://oppid.ru/,https://www.facebook.com/oppid.ru/,2_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Москва,"Некоммерческая организация Благотворительный фонд ""Столица Милосердия""",7702472455,2014-02-11,"г Москва, Мещанский р-н, ул Щепкина, д 9 стр 1",1,1,capital-mercy.ru,https://www.facebook.com/capitalmercy,1_9,...,1_7,2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Москва,"Российский общественный благотворительный фонд ветеранов (пенсионеров) войны, труда и Вооруженны...",7704058560,1998-06-05,"г Москва, р-н Соколиная Гора, ул Щербаковская, д 50/52",1,1,http://www.rbfondveteranov.ru/,NaN,1_4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Москва,"Благотворительный фонд поддержки и реализации программ в сфере культуры ""АВС""",7704278848,2011-09-30,"г Москва, р-н Хамовники, пер Знаменский Б., д 13 стр 2",0,0,http://www.avccharity.com/,NaN,4_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Drop empty rows
sites = dataset[~dataset["Ссылка на сайт"].isnull()]
sites.drop(columns=["Ссылка на соц сеть"], inplace=True)
social = dataset[~dataset["Ссылка на соц сеть"].isnull()]

# Merging social web-page to a web-site, adding new data samples 
social["Ссылка на сайт"] = social["Ссылка на соц сеть"]
social.drop(columns=["Ссылка на соц сеть"], inplace=True)
concat_dataset = pd.concat([sites, social], ignore_index=True)

/home/niwuro/Environments/WorkEnv/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/niwuro/Environments/WorkEnv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [29]:
concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(parser.link_splitter)
dataset.dropna(axis=0, subset=["Ссылка на сайт"], inplace=True)
concat_dataset["num_sites"] = concat_dataset["Ссылка на сайт"].apply(len)

In [30]:
# ! WARNING: This cell needs to be executed 3 times!
# TODO: 3 times-loop
max_len = len(concat_dataset)
for index, row in concat_dataset.iterrows():
    if len(row["Ссылка на сайт"])>1:
        sites = row["Ссылка на сайт"]
        last_site = sites.pop(-1)
        concat_dataset["Ссылка на сайт"].loc[index] = sites
        row["Ссылка на сайт"] = [last_site]
        concat_dataset.loc[max_len] = row
        max_len += 1

/home/niwuro/Environments/WorkEnv/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [31]:
concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(lambda lst: lst[0].strip())

In [32]:
# Fixing mistakes
concat_dataset["Ссылка на сайт"].loc[62] = "http://арктика-надежда.рф" 
concat_dataset["Ссылка на сайт"].loc[154] = "http://9911850.ru/"

/home/niwuro/Environments/WorkEnv/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [33]:
concat_dataset["web_text"] = concat_dataset["Ссылка на сайт"].apply(parser.get_text_contents)

/home/niwuro/Projects/NCO/npo_classification/parser.py:45: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 45 of the file /home/niwuro/Projects/NCO/npo_classification/parser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


KeyboardInterrupt: 

In [171]:
concat_dataset = concat_dataset.dropna(subset=["web_text"])
concat_dataset = concat_dataset[concat_dataset['web_text'].astype(str) != '[]']
concat_dataset = concat_dataset.reset_index().drop("index", axis=1)

In [174]:
concat_dataset["web_text"] = concat_dataset["web_text"].apply(lambda x: " ".join(x))

In [175]:
concat_dataset["web_text"]

0      function set_cookie(){var now = new Date();var time = now.getTime();time += 19360000 * 1000;now....
1      html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-tra...
2      html Благотворительный фонд Столица милосердия [if IE 6]>\n<link rel='stylesheet' type='text/css...
3      html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dt...
4      html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd" Благ...
5      html Контактная информация и реквизиты | Фонд Спивакова [if lt IE 9]>\n<script src="http://spiva...
6      html Главная | Я есть <script src="http://ajax.googleapis.com/ajax/libs/jquery/1.12.1/jquery.min...
7      html Фонд «Ковчег» | Поможем вместе window._wpemojiSettings = {"baseUrl":"http:\/\/s.w.org\/imag...
8      html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd" Áëàãîòâîð...
9      html if(!window.BX)window.BX={

In [176]:
concat_dataset.to_pickle(path="np_ngo_classification_links.pkl")

In [ ]:
concat_dataset["web_text"].loc[22]

In [6]:
mapping = pd.read_excel("np_ngo_mappings.xlsx")
mapping

In [7]:
dataset["links"] = dataset["Ссылка на сайт"].apply(parser.get_links)

/home/ubuntu/PycharmProjects/nko_classification/parser.py:16: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 16 of the file /home/ubuntu/PycharmProjects/nko_classification/parser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


In [40]:
with open('np_ngo_classification_links.pkl', 'rb') as f:
    df = pickle.load(f)

In [41]:
df.head()

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Вид деятельности 1,Вид деятельности 2,...,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12,num_sites,web_text
0,Москва,"Межрегиональная общественная благотворительная организация ""Общество защиты прав потребителей и ...",5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-й км, д 6",0,0,http://greenstartup.ru/,3_2,6_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,function set_cookie(){var now = new Date();var time = now.getTime();time += 19360000 * 1000;now....
1,Москва,"Межрегиональная благотворительная общественная организация инвалидов ""Общество пациентов с перви...",7702370774,2007-08-28,"г Москва, Мещанский р-н, ул Трифоновская, д 49, оф 41",1,1,http://oppid.ru/,2_1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"html PUBLIC ""-//W3C//DTD XHTML 1.0 Transitional//EN"" ""http://www.w3.org/TR/xhtml1/DTD/xhtml1-tra..."
2,Москва,"Некоммерческая организация Благотворительный фонд ""Столица Милосердия""",7702472455,2014-02-11,"г Москва, Мещанский р-н, ул Щепкина, д 9 стр 1",1,1,capital-mercy.ru,1_9,1_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,html Благотворительный фонд Столица милосердия [if IE 6]>\n<link rel='stylesheet' type='text/css...
3,Москва,"Российский общественный благотворительный фонд ветеранов (пенсионеров) войны, труда и Вооруженны...",7704058560,1998-06-05,"г Москва, р-н Соколиная Гора, ул Щербаковская, д 50/52",1,1,http://www.rbfondveteranov.ru/,1_4,5_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"html PUBLIC ""-//W3C//DTD XHTML 1.0 Strict//EN"" ""http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dt..."
4,Москва,"Благотворительный фонд ""Фонд Валерия Гергиева""",7709402778,2003-01-27,"г Москва, р-н Арбат, ул Молчановка Б., д 23 к 2",0,0,http://gergiev-charity.com/rus/,4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"html PUBLIC ""-//W3C//DTD HTML 4.01 Transitional//EN"" ""http://www.w3.org/TR/html4/loose.dtd"" Благ..."


In [52]:
test = parser.get_links('http://oppid.ru/')

In [54]:
test